In [1]:
# change the directory
import os 
os.chdir("c:\\Users\\jzou2\\Fall 2024-2025\\Project\\OptiMed")
from typing import TypedDict, List, Annotated
from operator import add


# import relevant packages
from agents.base_llm import OpenAILLMs
from agents.master import MasterWorkflow
from agents.user import UserLLM
from agents.diagnosis import DiagnosisLLM

# from agents.diagnosis_team.diagnosis_coordinator import DiagnosticCoordinator
# from agents.diagnosis_team.generalist import GeneralistLLM

In [3]:
class Report(TypedDict):
    """
    Defines the state for the multi-agent workflow
    Includes messages, patient input, and workflow states
    """
    patient_input: str
    messages: Annotated[List[dict], add]
    diagnosis: str
    recommendations: str
    appointment_details: str

mocked_report : Report = {'patient_input': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
    }


mock_patient_info = """
John Doe, a 45-year-old male, reports experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension diagnosed five years ago, managed with medication, and borderline high cholesterol with no current treatment. He has a family history of heart disease, as his father suffered a heart attack at age 50. He has no known allergies.
"""



#### Test LLMs and Agents

In [ ]:
# Pass
def test_base_llm():
    test_agent = OpenAILLMs()
    print(test_agent.prompt)
    test_agent_result = test_agent("What is the weather today")
    return test_agent_result
print(test_base_llm())

In [ ]:
# Pass
# def test_master_default():
#     master_agent = MasterWorkflow()
#     output = master_agent.test_call()
#     return output
# print(test_master_default())

def test_master_delegation():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report)
    return output
print(test_master_delegation())

In [ ]:
# Pass
def test_user():
    "my symptoms are shortness of breath and acute chest pain"
    "32"
    "i do not have any medical history"
    user_agent = UserLLM()
    output = user_agent()
    return output
print(test_user())

In [ ]:
def test_diagnose():
    diagnosis_agent = DiagnosisLLM()
    output = diagnosis_agent(mocked_report)
    return output
print(test_diagnose())

#### Test Synergies

In [ ]:
# FAILS TO GO TO DIAGOSIS
def master_to_user():
    master_agent = MasterWorkflow()
    output = master_agent.run()
    return output
print(master_to_user())

# PASS WITH STRICT PROMPT
def master_to_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report)
    return output
print(master_to_diagnosis())

def up_to_diagnosis():
    return None


{'patient_input': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.', 'messages': [], 'diagnosis': '', 'recommendations': '', 'appointment_details': ''}


> Entering new AgentExecutor chain...
Thought and Reasoning: The first step in completing the form is to fill out the **patient_input** field, which is already provided in the current report. Therefore, the next agent to delegate to is the **DIAGNOSIS** agent, as the patient input is complete and we need to determine the diagnosis based on that information.
Action: DIAGNOSIS
Final Answer: DIAGNOSIS

> Finished chain.


> Entering new AgentExecutor chain...
Thought and Reasoning: The patient, John Doe, is a 45-year-old male presenting with persistent chest pain, shortness of breath, occasional d

KeyboardInterrupt: 

#### Main Workflow

In [ ]:
# # creates class 
# final_workflow = MasterWorkflow() # called once

# # starts with initial form
# initiate = final_workflow()